# Evolver Loop 3 Analysis

## Critical Issue: Both submissions failed with 'Overlapping trees in group 008'

We need to:
1. Understand the EXACT overlap detection the LB uses
2. Investigate group 008 specifically
3. Find a truly valid submission

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore')

print('Libraries loaded')

Libraries loaded


In [2]:
# Tree geometry constants
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

BASE_TREE_VERTICES = np.array([
    [0.0, TIP_Y],
    [TOP_W/2, TIER_1_Y],
    [TOP_W/4, TIER_1_Y],
    [MID_W/2, TIER_2_Y],
    [MID_W/4, TIER_2_Y],
    [BASE_W/2, BASE_Y],
    [TRUNK_W/2, BASE_Y],
    [TRUNK_W/2, TRUNK_BOTTOM_Y],
    [-TRUNK_W/2, TRUNK_BOTTOM_Y],
    [-TRUNK_W/2, BASE_Y],
    [-BASE_W/2, BASE_Y],
    [-MID_W/4, TIER_2_Y],
    [-MID_W/2, TIER_2_Y],
    [-TOP_W/4, TIER_1_Y],
    [-TOP_W/2, TIER_1_Y],
])

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    rotation_matrix = np.array([[cos_a, -sin_a], [sin_a, cos_a]])
    rotated = BASE_TREE_VERTICES @ rotation_matrix.T
    translated = rotated + np.array([x, y])
    return Polygon(translated)

def parse_submission(df):
    result = df.copy()
    for col in ['x', 'y', 'deg']:
        result[col] = result[col].str.replace('s', '').astype(float)
    return result

print('Helper functions defined')

Helper functions defined


In [3]:
# LB-style overlap check: intersects() and not touches()
def check_lb_overlaps(polygons):
    """Check for overlaps using LB's method: intersects() and not touches()"""
    overlaps = []
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                overlaps.append((i, j))
    return overlaps

# Also check with area-based method for comparison
def check_area_overlaps(polygons):
    """Check for overlaps using area > 0 method"""
    overlaps = []
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            intersection = polygons[i].intersection(polygons[j])
            if intersection.area > 0:
                overlaps.append((i, j, intersection.area))
    return overlaps

print('Overlap check functions defined')

Overlap check functions defined


In [4]:
# Load the current submission and check group 008 specifically
print('Loading current submission...')
current_df = pd.read_csv('/home/submission/submission.csv')
current_parsed = parse_submission(current_df)
print(f'Shape: {current_parsed.shape}')

# Check group 008
prefix = '008_'
config_df = current_parsed[current_parsed['id'].str.startswith(prefix)]
print(f'\nGroup 008 has {len(config_df)} trees')
print(config_df)

Loading current submission...
Shape: (20100, 4)

Group 008 has 8 trees
       id         x         y         deg
28  008_0 -0.249562 -0.411092   51.766738
29  008_1  0.664668 -0.857305  113.629378
30  008_2 -0.664668  0.257305  293.629378
31  008_3  0.557305  0.326837  203.629378
32  008_4  0.249562 -0.188908  231.766738
33  008_5 -0.206589 -0.855262  293.629378
34  008_6  0.206589  0.255262  113.629378
35  008_7 -0.557305 -0.926837   23.629378


In [5]:
# Create polygons for group 008 and check overlaps
polygons_008 = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]

lb_overlaps = check_lb_overlaps(polygons_008)
area_overlaps = check_area_overlaps(polygons_008)

print(f'LB-style overlaps in group 008: {len(lb_overlaps)}')
if lb_overlaps:
    print(f'Overlapping pairs: {lb_overlaps}')

print(f'\nArea-based overlaps in group 008: {len(area_overlaps)}')
if area_overlaps:
    for i, j, area in area_overlaps:
        print(f'  Trees {i} and {j}: area = {area}')

LB-style overlaps in group 008: 0

Area-based overlaps in group 008: 0


In [6]:
# Check ALL groups with LB-style overlap detection
print('Checking ALL groups with LB-style overlap detection...')

lb_overlap_groups = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = current_parsed[current_parsed['id'].str.startswith(prefix)]
    if len(config_df) != n:
        print(f'WARNING: Group {n} has {len(config_df)} trees instead of {n}')
        continue
    
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    overlaps = check_lb_overlaps(polygons)
    if overlaps:
        lb_overlap_groups.append((n, len(overlaps)))

print(f'\nGroups with LB-style overlaps: {len(lb_overlap_groups)}')
if lb_overlap_groups:
    print('First 20 groups with overlaps:')
    for n, count in lb_overlap_groups[:20]:
        print(f'  Group {n:03d}: {count} overlapping pairs')

Checking ALL groups with LB-style overlap detection...



Groups with LB-style overlaps: 0


In [7]:
# Check chistyakov_best.csv - the evaluator says this has 0 overlaps
print('\nChecking chistyakov_best.csv...')
chistyakov_df = pd.read_csv('/home/code/preoptimized/chistyakov_best.csv')
chistyakov_parsed = parse_submission(chistyakov_df)

# Check group 008 in chistyakov
prefix = '008_'
config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith(prefix)]
polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
overlaps = check_lb_overlaps(polygons)
print(f'Group 008 in chistyakov_best.csv: {len(overlaps)} LB-style overlaps')

# Check ALL groups
chistyakov_overlap_groups = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith(prefix)]
    if len(config_df) != n:
        continue
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    overlaps = check_lb_overlaps(polygons)
    if overlaps:
        chistyakov_overlap_groups.append((n, len(overlaps)))

print(f'Total groups with LB-style overlaps in chistyakov_best.csv: {len(chistyakov_overlap_groups)}')
if chistyakov_overlap_groups:
    print(f'Groups: {chistyakov_overlap_groups}')


Checking chistyakov_best.csv...
Group 008 in chistyakov_best.csv: 0 LB-style overlaps


Total groups with LB-style overlaps in chistyakov_best.csv: 0


In [8]:
# The LB is rejecting with "Overlapping trees in group 008" but our check shows 0 overlaps
# This could mean:
# 1. The LB uses different tree geometry
# 2. The LB uses different precision
# 3. There's a floating point issue in how we read the CSV

# Let's check the raw CSV values for group 008
print('Raw CSV values for group 008:')
raw_df = pd.read_csv('/home/submission/submission.csv')
group_008 = raw_df[raw_df['id'].str.startswith('008_')]
print(group_008)

# Check if there are any precision issues
print('\nParsed values:')
for _, row in group_008.iterrows():
    x = float(row['x'].replace('s', ''))
    y = float(row['y'].replace('s', ''))
    deg = float(row['deg'].replace('s', ''))
    print(f"{row['id']}: x={x:.20f}, y={y:.20f}, deg={deg:.20f}")

Raw CSV values for group 008:
       id                      x                     y                  deg
28  008_0  s-0.24956221975123125  s-0.4110915693917769    s51.7667378176506
29  008_1    s0.6646675428904323  s-0.8573053542961684  s113.62937778579037
30  008_2   s-0.6646675428602861   s0.2573053553705201   s293.6293777306379
31  008_3     s0.557305355195559   s0.3268368934018002  s203.62937773065627
32  008_4    s0.2495622198407891  s-0.1889084299202516   s231.7667378027784
33  008_5   s-0.2065888753430593  s-0.8552621014916894   s293.6293777857903
34  008_6   s0.20658887514113536   s0.2552621014870126  s113.62937773063783
35  008_7   s-0.5573053551764364  s-0.9268368933098561  s23.629377730657012

Parsed values:
008_0: x=-0.24956221975123124568, y=-0.41109156939177687873, deg=51.76673781765060056159
008_1: x=0.66466754289043228354, y=-0.85730535429616838705, deg=113.62937778579036773863
008_2: x=-0.66466754286028606469, y=0.25730535537052007244, deg=293.62937773063788426953
008

In [9]:
# Let's check what the LB might be using for tree geometry
# Maybe the LB uses a slightly different tree definition

# Let's also check if there's a buffer/tolerance issue
# Try checking with a small buffer around polygons

print('Checking with buffered polygons (simulating potential LB tolerance)...')

for buffer_size in [1e-10, 1e-9, 1e-8, 1e-7, 1e-6]:
    config_df = current_parsed[current_parsed['id'].str.startswith('008_')]
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']).buffer(buffer_size) for _, row in config_df.iterrows()]
    overlaps = check_lb_overlaps(polygons)
    print(f'Buffer {buffer_size}: {len(overlaps)} overlaps')
    if overlaps:
        print(f'  Pairs: {overlaps}')

Checking with buffered polygons (simulating potential LB tolerance)...
Buffer 1e-10: 10 overlaps
  Pairs: [(0, 5), (0, 6), (0, 7), (1, 5), (2, 6), (3, 4), (3, 6), (4, 5), (4, 6), (5, 7)]
Buffer 1e-09: 10 overlaps
  Pairs: [(0, 5), (0, 6), (0, 7), (1, 5), (2, 6), (3, 4), (3, 6), (4, 5), (4, 6), (5, 7)]
Buffer 1e-08: 10 overlaps
  Pairs: [(0, 5), (0, 6), (0, 7), (1, 5), (2, 6), (3, 4), (3, 6), (4, 5), (4, 6), (5, 7)]
Buffer 1e-07: 10 overlaps
  Pairs: [(0, 5), (0, 6), (0, 7), (1, 5), (2, 6), (3, 4), (3, 6), (4, 5), (4, 6), (5, 7)]
Buffer 1e-06: 10 overlaps
  Pairs: [(0, 5), (0, 6), (0, 7), (1, 5), (2, 6), (3, 4), (3, 6), (4, 5), (4, 6), (5, 7)]


In [10]:
# The trees are exactly touching! The LB might use slightly different precision
# Let's check the actual distance between polygons

print('Checking actual distances between polygons in group 008...')
config_df = current_parsed[current_parsed['id'].str.startswith('008_')]
polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]

for i in range(len(polygons)):
    for j in range(i+1, len(polygons)):
        dist = polygons[i].distance(polygons[j])
        intersects = polygons[i].intersects(polygons[j])
        touches = polygons[i].touches(polygons[j])
        if dist < 1e-6:  # Very close
            print(f'Trees {i} and {j}: distance={dist:.2e}, intersects={intersects}, touches={touches}')

Checking actual distances between polygons in group 008...
Trees 0 and 5: distance=1.65e-15, intersects=False, touches=False
Trees 0 and 6: distance=6.06e-16, intersects=False, touches=False
Trees 0 and 7: distance=1.87e-10, intersects=False, touches=False
Trees 1 and 5: distance=1.22e-16, intersects=False, touches=False
Trees 2 and 6: distance=3.53e-16, intersects=False, touches=False
Trees 3 and 4: distance=4.01e-15, intersects=False, touches=False
Trees 3 and 6: distance=1.19e-14, intersects=False, touches=False
Trees 4 and 5: distance=9.23e-16, intersects=False, touches=False
Trees 4 and 6: distance=1.56e-15, intersects=False, touches=False
Trees 5 and 7: distance=1.08e-15, intersects=False, touches=False


In [11]:
# Check chistyakov_best.csv group 008 distances
print('Checking distances in chistyakov_best.csv group 008...')
config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith('008_')]
polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]

close_pairs = 0
for i in range(len(polygons)):
    for j in range(i+1, len(polygons)):
        dist = polygons[i].distance(polygons[j])
        if dist < 1e-6:  # Very close
            close_pairs += 1
            print(f'Trees {i} and {j}: distance={dist:.2e}')

print(f'\nTotal close pairs (< 1e-6): {close_pairs}')

Checking distances in chistyakov_best.csv group 008...
Trees 0 and 1: distance=3.86e-09
Trees 0 and 3: distance=4.33e-07
Trees 1 and 3: distance=2.06e-07
Trees 1 and 5: distance=7.76e-08
Trees 1 and 6: distance=4.59e-08
Trees 2 and 6: distance=1.17e-08
Trees 3 and 4: distance=4.13e-08
Trees 4 and 7: distance=3.67e-07

Total close pairs (< 1e-6): 8


In [12]:
# Check chistyakov_best.csv with buffer
print('Checking chistyakov_best.csv group 008 with buffer...')
config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith('008_')]

for buffer_size in [1e-10, 1e-9, 1e-8, 1e-7, 1e-6]:
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']).buffer(buffer_size) for _, row in config_df.iterrows()]
    overlaps = check_lb_overlaps(polygons)
    print(f'Buffer {buffer_size}: {len(overlaps)} overlaps')

Checking chistyakov_best.csv group 008 with buffer...
Buffer 1e-10: 0 overlaps
Buffer 1e-09: 0 overlaps
Buffer 1e-08: 2 overlaps
Buffer 1e-07: 5 overlaps
Buffer 1e-06: 8 overlaps


In [ ]:
# Let's check ALL groups in chistyakov_best.csv with a 1e-10 buffer
# to see if it would pass the LB

print('Checking ALL groups in chistyakov_best.csv with 1e-10 buffer...')
buffer_size = 1e-10
overlap_groups = []

for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith(prefix)]
    if len(config_df) != n:
        continue
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']).buffer(buffer_size) for _, row in config_df.iterrows()]
    overlaps = check_lb_overlaps(polygons)
    if overlaps:
        overlap_groups.append((n, len(overlaps)))

print(f'Groups with overlaps (buffer 1e-10): {len(overlap_groups)}')
if overlap_groups:
    print(f'Groups: {overlap_groups[:20]}')